In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

options = uc.ChromeOptions()

options.add_experimental_option('prefs', {
    "download.default_directory": "correspondence",  # Change this to your desired directory
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # Force Chrome to download PDFs
})
# Initialize the Chrome webdriver
#driver = webdriver.Chrome()
driver = uc.Chrome(use_subprocess=False, options=options)

# Go to the webpage
driver.get("https://committees.parliament.uk/committee/127/public-accounts-committee/publications/3/correspondence/")


page_number = 1

while True:
    # Create a DataFrame to store the scraped data
    df = pd.DataFrame(columns=['Title', 'Topic', 'Date','File Link'])

    # Wait for the page to load and the correspondence entries to be present
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "card-inner")]')))

    # Get all the correspondence entries
    entries = driver.find_elements(By.XPATH, '//div[contains(@class, "card-inner")]')

    for i, entry in enumerate(entries):
        # Get the title
        title = entry.find_element(By.XPATH, './/div[contains(@class, "primary-info")]').text

        # Get the topic if it exists
        topic = None
        topic_element = entry.find_elements(By.XPATH, './/div[contains(@class, "item")]')
        if topic_element:
            topic = topic_element[0].text

        date = entry.find_element(By.XPATH, './/div[contains(@class, "indicators-left")]').text

        # Get the file link
        try:
            file_link = entry.find_element(By.XPATH, './/a[contains(@class, "dropdown-item")]').get_attribute('href')
        except:
            file_link = None

        # Add the data to the DataFrame
        df.loc[i] = [title, topic, date, file_link]


    # Save the DataFrame to a CSV file
    df.to_csv('pac_scrapes/' + f'scraped_data_page_{page_number}.csv', index=False)

    # Check if there's a next page
    next_page = driver.find_elements(By.XPATH, '//li[contains(@class, "next")]')
    if not next_page:
        break

    # Navigate to the next page
    next_page[0].click()

    # Increment the page number
    page_number += 1

driver.quit()


In [66]:
import os
from datetime import datetime

folder = 'pac_scrapes'

# Get list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]
all_dfs = []

# Sort by last modified time in descending order 
csv_files.sort(key=lambda f: os.path.getmtime(os.path.join(folder, f)), reverse=True) 
for file in reversed(csv_files):
    df = pd.read_csv(folder + '/' + file)
    all_dfs.append(df)

combined = pd.concat(all_dfs, ignore_index=True)
combined

,Title,Topic,Date,File Link
0,"Correspondence from Dame Bernadette Kelly DCB,...",Inquiry Active travel in England,Published 19 July 2023,https://committees.parliament.uk/publications/...
1,"Correspondence from Sarah Healey CB CV, Perman...",Inquiry Supported housing,Published 11 July 2023,https://committees.parliament.uk/publications/...
2,"Correspondence from Tim Hollingsworth, Chief E...",NaN,Published 11 July 2023,https://committees.parliament.uk/publications/...
3,"Correspondence from Alex Chisholm, Civil Servi...",NaN,Published 11 July 2023,https://committees.parliament.uk/publications/...
4,"Correspondence from James Armstrong, Managing ...",Inquiry Decarbonising the power sector,Published 11 July 2023,https://committees.parliament.uk/publications/...
...,...,...,...,...
5918,"Correspondence from Francis Maude MP, Danny Al...",NaN,Published 11 June 2014,https://www.parliament.uk/globalassets/documen...
5919,Correspondence from Cabinet Office and HM Trea...,NaN,Published 11 June 2014,https://www.parliament.uk/globalassets/documen...
5920,Correspondence from Sir Bob Kerslake to Chair ...,NaN,Published 3 April 2014,https://www.parliament.uk/globalassets/documen...
5921,Correspondence from HM Revenue & Customs to Ch...,NaN,Published 3 April 2014,https://www.parliament.uk/globalassets/documen...


In [80]:
options = uc.ChromeOptions()
import time
download_folder = "/correspondence_pdf"

options.add_experimental_option('prefs', {
    "download.default_directory":download_folder ,  # Change this to your desired directory
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # Force Chrome to download PDFs
})
# Initialize the Chrome webdriver
#driver = webdriver.Chrome()

# Go to the webpage

for index, row in combined.iterrows():
    options = uc.ChromeOptions()
    name = row['Title'][0:50] + row['Date']

    count = 1
    while os.path.exists(download_folder + '/' + name):
        name = name + "_"+ str(count)
        count += 1

    options.add_experimental_option('prefs', {
    "download.default_directory":download_folder + '/' + name ,  # Change this to your desired directory
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True })

    # Construct file name 

    os.mkdir(download_folder + '/' + name)

    driver = uc.Chrome(use_subprocess=False, options=options, headless=True)

    # Navigate to file url 
    driver.get(row['File Link'])
    time.sleep(3)
    driver.quit()
    combined.at[index, 'download_loc'] =  name
    


InvalidArgumentException: Message: invalid argument: missing command parameters
Stacktrace:
0   undetected_chromedriver             0x0000000102e066b8 undetected_chromedriver + 4937400
1   undetected_chromedriver             0x0000000102dfdb73 undetected_chromedriver + 4901747
2   undetected_chromedriver             0x00000001029bb616 undetected_chromedriver + 435734
3   undetected_chromedriver             0x0000000102a1e9b0 undetected_chromedriver + 842160
4   undetected_chromedriver             0x0000000102a1e2e7 undetected_chromedriver + 840423
5   undetected_chromedriver             0x00000001029852da undetected_chromedriver + 213722
6   undetected_chromedriver             0x0000000102dc22ad undetected_chromedriver + 4657837
7   undetected_chromedriver             0x0000000102dc7130 undetected_chromedriver + 4677936
8   undetected_chromedriver             0x0000000102dcddef undetected_chromedriver + 4705775
9   undetected_chromedriver             0x0000000102dc805a undetected_chrom

In [82]:
combined.to_csv('pac_scrapped_by_folder.csv')